In [2]:
import numpy as np
import pandas as pd 

import os

In [8]:
train = 'C:\Users\Aayush\Downloads\Birdspecies\train'
validation = 'C:\Users\Aayush\Downloads\Birdspecies\valid'
test = 'C:\Users\Aayush\Downloads\Birdspecies\test'
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input,Flatten, Dropout,GlobalMaxPooling2D,Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
import tensorflow as tf

In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255)#initialize train generator 
                                 


valid_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize validation generator 

test_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize validation generator 
train_generator = train_datagen.flow_from_directory(train, target_size=(224,224),batch_size=32,class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(validation, target_size=(224,224),batch_size=32,class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test, target_size=(224,224),batch_size=32,class_mode='categorical')

Found 70626 images belonging to 450 classes.
Found 2250 images belonging to 450 classes.
Found 2250 images belonging to 450 classes.


In [9]:
inception = tf.keras.applications.InceptionV3(weights='imagenet',include_top=False,input_shape=(224,224,3))

2022-11-07 09:58:50.305560: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 09:58:50.395044: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 09:58:50.395795: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 09:58:50.398033: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

87924736/87910968 [==============================] - 4s 0us/step


In [12]:
inception.trainable = True
for layer in inception.layers[:165]:
    layer.trainable = False 

In [13]:
for idx, layer in enumerate(inception.layers):
    print(f' {idx}:  {layer.name}: trainable = {layer.trainable}')

 0:  input_1: trainable = False
 1:  conv2d: trainable = False
 2:  batch_normalization: trainable = False
 3:  activation: trainable = False
 4:  conv2d_1: trainable = False
 5:  batch_normalization_1: trainable = False
 6:  activation_1: trainable = False
 7:  conv2d_2: trainable = False
 8:  batch_normalization_2: trainable = False
 9:  activation_2: trainable = False
 10:  max_pooling2d: trainable = False
 11:  conv2d_3: trainable = False
 12:  batch_normalization_3: trainable = False
 13:  activation_3: trainable = False
 14:  conv2d_4: trainable = False
 15:  batch_normalization_4: trainable = False
 16:  activation_4: trainable = False
 17:  max_pooling2d_1: trainable = False
 18:  conv2d_8: trainable = False
 19:  batch_normalization_8: trainable = False
 20:  activation_8: trainable = False
 21:  conv2d_6: trainable = False
 22:  conv2d_9: trainable = False
 23:  batch_normalization_6: trainable = False
 24:  batch_normalization_9: trainable = False
 25:  activation_6: trainab

In [14]:
#get the last layer    
last_layer = inception.get_layer('mixed8')

print('last layer output shape: ', last_layer.output_shape)

layer_output = last_layer.output

last layer output shape:  (None, 5, 5, 1280)


In [15]:
n_categories = len(os.listdir('/kaggle/input/100-bird-species/train'))# number of categories
print(n_categories)

450


In [16]:
# x  = BatchNormalization()(layer_output)
x = Flatten()(layer_output)
x = Dense(1024,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.4)(x)
x = Dense(n_categories, activation='softmax')(x)

model = Model(inputs=inception.inputs, outputs=x)

In [17]:
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [18]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20)

2022-11-07 10:09:41.874621: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-11-07 10:09:47.686357: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


2208/2208 [==============================] - 635s 282ms/step - loss: 3.0430 - accuracy: 0.5337 - val_loss: 1.0540 - val_accuracy: 0.8978
Epoch 2/20
2208/2208 [==============================] - 213s 96ms/step - loss: 1.1072 - accuracy: 0.8547 - val_loss: 0.7411 - val_accuracy: 0.9244
Epoch 3/20
2208/2208 [==============================] - 209s 94ms/step - loss: 0.7062 - accuracy: 0.9117 - val_loss: 0.5815 - val_accuracy: 0.9329
Epoch 4/20
2208/2208 [==============================] - 226s 102ms/step - loss: 0.5221 - accuracy: 0.9346 - val_loss: 0.4957 - val_accuracy: 0.9347
Epoch 5/20
2208/2208 [==============================] - 208s 94ms/step - loss: 0.4241 - accuracy: 0.9443 - val_loss: 0.4251 - val_accuracy: 0.9391
Epoch 6/20
2208/2208 [==============================] - 206s 93ms/step - loss: 0.3668 - accuracy: 0.9514 - val_loss: 0.4069 - val_accuracy: 0.9449
Epoch 7/20
2208/2208 [==============================] - 214s 97ms/step - loss: 0.3240 - accuracy: 0.9556 - val_loss: 0.4035 - v

In [19]:
model.evaluate(test_generator)


71/71 [==============================] - 20s 283ms/step - loss: 0.2904 - accuracy: 0.9556


[0.290367990732193, 0.9555555582046509]

In [21]:
filename = "test_birds_model.tflite"
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()
open(filename, "wb").write(tflite_model)

2022-11-07 11:30:19.036833: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 11:30:19.037235: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-11-07 11:30:19.037354: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-11-07 11:30:19.037896: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 11:30:19.038323: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 11:30:19.038651: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

175574844

In [22]:
model.save("final_model.h5")